In [34]:
from torchmetrics.multimodal.clip_score import CLIPScore
from functools import partial
from PIL import Image
import torch
import os



In [36]:


def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    images_int = np.expand_dims(images_int, axis=0)  # Add batch dimension
    score = metrics(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(score), 4)


In [3]:
def process_images_and_captions(image_paths, captions):
    for img_path, caption in zip(image_paths, captions):
        img = Image.open(img_path)
        score = calculate_clip_score(img, caption)
        print(f"Image: {img_path}, Caption: {caption}, CLIP Score: {score}")


'/home/reutsalman/repo/Stable-Diffusion-Attribute-Binding'

Data set & captions


In [31]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import os
import requests
import numpy as np


USER_AGENT = get_datasets_user_agent()

In [8]:
num_threads = 20
dset = load_dataset("google-research-datasets/conceptual_captions",streaming=True)


In [9]:
dset_iter = iter(dset['train'])

In [23]:

def conceptual_captioning_stream_to_dir (save_dir, image_num):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    captions = []
    for i in range(image_num):
        item2 = next(dset_iter)
        image_url = item2['image_url']
        caption = item2['caption']
        response = requests.get(image_url, stream=True)
        try:
            if response.status_code == 200:
                image_path = os.path.join(save_dir, f"image_{i}.jpg")
                with open(image_path, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                captions.append(caption)
            
            else:
                print(f"Failed to download image {i} from {image_url}")
        except Exception as e:
            print(f"Error downloading image {i} from {image_url}: {e}")
    return captions , save_dir

In [37]:
def evaluation(save_dir, captions , metric= "clip"):
    if (metric == "clip"):
        print(f"Metric used is CLIP Score")
        metrics = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
    image_files = sorted(os.listdir(save_dir))  # Ensure the images are processed in order
    for img_file, caption in zip(image_files, captions):
        img_path = os.path.join(save_dir, img_file)
        if img_path.endswith(".jpg") or img_path.endswith(".png"):
            img = Image.open(img_path)
            img_array = np.array(img) / 255.0
            score = calculate_clip_score(img_array, caption)
            print(f"Image: {img_path}, Caption: {caption}, CLIP Score: {score}")

In [27]:
captions = []
path_dir = os.path.join(os.getcwd(), "conceptual_captioning")
captions, save_dir = conceptual_captioning_stream_to_dir(path_dir,image_num=20)


the sign promises as much as the glorious blue sky . added to list
Failed to download image 1 from https://media.gettyimages.com/photos/architectural-details-of-a-bridge-picture-id511832249?s=612x612
people tour and enjoy the public park during summer added to list
interesting 1930 's poster for a cosmetic company with stores . added to list
racecar driver steers his car during video game subject . added to list
vintage elegant floral card with frame decorated with black and white lilies on a pink background . added to list
heavy snow falls over a snow lined river . added to list
bright living room in the attic added to list
Failed to download image 8 from https://media.gettyimages.com/photos/actor-kevin-mchale-attends-the-3rd-annual-nautica-oceana-beach-house-picture-id472759470?s=612x612
Failed to download image 9 from https://thumb1.shutterstock.com/display_pic_with_logo/1448648/675197230/stock-vector-illustration-of-a-long-shadow-australia-map-its-flag-and-a-comic-balloon-with-a-so

In [28]:
for caption in captions:
    print(caption)

the sign promises as much as the glorious blue sky .
people tour and enjoy the public park during summer
interesting 1930 's poster for a cosmetic company with stores .
racecar driver steers his car during video game subject .
vintage elegant floral card with frame decorated with black and white lilies on a pink background .
heavy snow falls over a snow lined river .
bright living room in the attic
green sea turtle isolated on a white background 3d illustration
person , was surprised by the staff
red and white flag on the mast
football player celebrates scoring for football team against football team in the final
concept plug - in hybrid car on display
a pencil drawing of a zebra and her baby .


In [38]:
evaluation(save_dir,captions)

Metric used is CLIP Score


/home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Image: /home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/conceptual_captioning/image_0.jpg, Caption: the sign promises as much as the glorious blue sky ., CLIP Score: 25.6883
Image: /home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/conceptual_captioning/image_11.jpg, Caption: people tour and enjoy the public park during summer, CLIP Score: 17.9286
Image: /home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/conceptual_captioning/image_12.jpg, Caption: interesting 1930 's poster for a cosmetic company with stores ., CLIP Score: 6.0892
Image: /home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/conceptual_captioning/image_13.jpg, Caption: racecar driver steers his car during video game subject ., CLIP Score: 21.27
Image: /home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/conceptual_captioning/image_14.jpg, Caption: vintage elegant floral card with frame decorated with black and white lilies on a pink background ., CLIP Score: 1.942
Image: /home/reutsalman/repo/

UnidentifiedImageError: cannot identify image file '/home/reutsalman/repo/Stable-Diffusion-Attribute-Binding/conceptual_captioning/image_7.jpg'